In [57]:
import pandas as pd
import json
file_path='/home/daisy/mydrive/web-dev/deep_learning_pipeline-train-inference-/training_controller/apis/configs/tf2/centernet_hourglass104_1024x1024_coco17_tpu-32.config'
path_2='/home/daisy/mydrive/web-dev/deep_learning_pipeline-train-inference-/kk.config'

In [67]:
import argparse

import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2


pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
print(pipeline_config)
with tf.io.gfile.GFile(file_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 
pipeline_config.train_input_config.label_map_path= "123/label_map.txt"
pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.height = 300                                                                                                                                                                                          
pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.width = 300

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(path_2, "wb") as f:                                                                                                                                                                                                                       
    f.write(config_text)  

AttributeError: train_input_config

In [ ]:
# def parse_arguments():                                                                                                                                                                                                                                                
#     parser = argparse.ArgumentParser(description='')                                                                                                                                                                                                                  
#     parser.add_argument('pipeline')                                                                                                                                                                                                                                   
#     parser.add_argument('output')                                                                                                                                                                                                                                     
#     return parser.parse_args()                                                                                                                                                                                                                                        


def main():                                                                                                                                                                                                                                                           
    args = parse_arguments()                                                                                                                                                                                                                                          
                                                                                                                                                                                                             

    with tf.gfile.GFile(args.pipeline, "r") as f:                                                                                                                                                                                                                     
        proto_str = f.read()                                                                                                                                                                                                                                          
        text_format.Merge(proto_str, pipeline_config)                                                                                                                                                                                                                 

    pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.height = 300                                                                                                                                                                                          
    pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.width = 300                                                                                                                                                                                           

    config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
    with tf.gfile.Open(args.output, "wb") as f:                                                                                                                                                                                                                       
        f.write(config_text)                                                                                                                                                                                                                                          


if __name__ == '__main__':                                                                                                                                                                                                                                            
    main()

In [59]:
from object_detection.utils import config_util

pipeline_config = config_util.get_configs_from_pipeline_file(file_path)

In [68]:
pipeline_config

model {
  center_net {
    num_classes: 90
    feature_extractor {
      type: "hourglass_104"
      channel_means: 104.01361846923828
      channel_means: 114.03422546386719
      channel_means: 119.91659545898438
      channel_stds: 73.60276794433594
      channel_stds: 69.89082336425781
      channel_stds: 70.91507720947266
      bgr_ordering: true
    }
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 1024
        max_dimension: 1024
        pad_to_max_dimension: true
      }
    }
    object_detection_task {
      task_loss_weight: 1.0
      offset_loss_weight: 1.0
      scale_loss_weight: 0.10000000149011612
      localization_loss {
        l1_localization_loss {
        }
      }
    }
    object_center_params {
      object_center_loss_weight: 1.0
      classification_loss {
        penalty_reduced_logistic_focal_loss {
          alpha: 2.0
          beta: 4.0
        }
      }
      min_box_overlap_iou: 0.699999988079071
      max_box_predictions: 

In [41]:
import argparse

import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2


# def parse_arguments():                                                                                                                                                                                                                                                
#     parser = argparse.ArgumentParser(description='')                                                                                                                                                                                                                  
#     parser.add_argument('pipeline')                                                                                                                                                                                                                                   
#     parser.add_argument('output')                                                                                                                                                                                                                                     
#     return parser.parse_args()                                                                                                                                                                                                                                        


def main():
    import pandas as pd
    import json
    file_path='/home/daisy/mydrive/web-dev/deep_learning_pipeline-train-inference-/training_controller/apis/configs/tf2/centernet_hourglass104_1024x1024_coco17_tpu-32.config'                                                                                                                                                                                                                                         
    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          

    with tf.gfile.GFile(args.pipeline, "r") as f:                                                                                                                                                                                                                     
        proto_str = f.read()                                                                                                                                                                                                                                          
        text_format.Merge(proto_str, pipeline_config)                                                                                                                                                                                                                 

    pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.height = 300                                                                                                                                                                                          
    pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.width = 300                                                                                                                                                                                           

    config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
    with tf.gfile.Open(args.output, "wb") as f:                                                                                                                                                                                                                       
        f.write(config_text)                                                                                                                                                                                                                                          


if __name__ == '__main__':                                                                                                                                                                                                                                            
    main() 

AttributeError: module 'tensorflow' has no attribute 'gfile'

In [39]:
TOOL_DIR=tool/tf-models/research

(
   cd $TOOL_DIR
   protoc object_detection/protos/*.proto --python_out=.
)

export PYTHONPATH=$PYTHONPATH:$TOOL_DIR:$TOOL_DIR/slim

python3 edit_pipeline.py pipeline.config pipeline_new.config

SyntaxError: invalid syntax (<ipython-input-39-ae70657cc7c2>, line 4)

In [38]:
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = '/tensorflow/models/data/train100.record'

AttributeError: 'dict' object has no attribute 'train_input_reader'

In [2]:

from protoc_gen_validate.validator import validate, ValidationFailed

p = Person(first_name="Foo", last_name="Bar", age=42)
try:
    validate(p)
except ValidationFailed as err:
    print(err)

ModuleNotFoundError: No module named 'protoc_gen_validate'

In [5]:
from __future__ import absolute_import
from __future__ import division

from __future__ import print_function

import logging
from typing import Any, Iterable, List, Optional, Set, Text, Tuple, Union

from tensorflow_data_validation import types
from tensorflow_data_validation.utils import io_util

from google.protobuf import descriptor
from google.protobuf import text_format
from tensorflow_metadata.proto.v0 import schema_pb2

In [4]:
!pip install tensorflow-data-validation

     |████████████████████████████████| 1.4 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 454.4 MB 46 kB/s  eta 0:00:01    |██                              | 27.6 MB 6.3 MB/s eta 0:01:08     |███████████████████▌            | 277.2 MB 5.1 MB/s eta 0:00:35     |████████████████████▉           | 295.3 MB 6.1 MB/s eta 0:00:27
     |████████████████████████████████| 11.5 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 5.4 MB/s eta 0:00:01
  Using cached pyarrow-2.0.0-cp38-cp38-manylinux2014_x86_64.whl (17.8 MB)
     |████████████████████████████████| 48 kB 794 kB/s eta 0:00:01
     |████████████████████████████████| 134 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 144 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 173 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 435 kB 5.8 MB/s eta 0:00:01
     |████████████████████████████████|

     |████████████████████████████████| 1.2 MB 5.2 MB/s eta 0:00:01
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
     |████████████████████████████████| 462 kB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 5.2 MB/s eta 0:00:01
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached numpy-1.19.5-cp38-cp38-manylinux2010_x86_64.whl (14.9 MB)
  Using cached tensorboard_data_server-0.6.1-py3-none-manylinux2010_x86_64.whl (4.9 MB)


  Created wheel for google-apitools: filename=google_apitools-0.5.31-py3-none-any.whl size=131042 sha256=bfc9e99a0a69f7c8f914bcbe6f72946dc6df152410cd47731dfc181e01629a5c
  Stored in directory: /home/daisy/.cache/pip/wheels/d7/54/79/85de1824f2f4175fb4960c72afb10045d86700c3941dc73685
  Created wheel for grpc-google-iam-v1: filename=grpc_google_iam_v1-0.12.3-py3-none-any.whl size=18482 sha256=fffb1c1a629b9776a0bea208115af836fd681fa8dc44d5279ef61b8aeb37b744
  Stored in directory: /home/daisy/.cache/pip/wheels/8f/b9/13/fce3d62261f63c01b28281fe6a9d704a7af65d96ff2c88552e
Successfully built google-apitools grpc-google-iam-v1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Attempting uni

In [ ]:
def get_feature(schema: schema_pb2.Schema,
                feature_path: Union[types.FeatureName, types.FeaturePath]
               ) -> schema_pb2.Feature:
  """Get a feature from the schema.
  Args:
    schema: A Schema protocol buffer.
    feature_path: The path of the feature to obtain from the schema. If a
      FeatureName is passed, a one-step FeaturePath will be constructed and
      used. For example, "my_feature" -> types.FeaturePath(["my_feature"])
  Returns:
    A Feature protocol buffer.
  Raises:
    TypeError: If the input schema is not of the expected type.
    ValueError: If the input feature is not found in the schema.
  """
  if not isinstance(schema, schema_pb2.Schema):
    raise TypeError('schema is of type %s, should be a Schema proto.' %
                    type(schema).__name__)

  if not isinstance(feature_path, types.FeaturePath):
    feature_path = types.FeaturePath([feature_path])

  feature_container = schema.feature
  parent = feature_path.parent()
  if parent:
    for step in parent.steps():
      f = look_up_feature(step, feature_container)
      if f is None:
        raise ValueError('Feature %s not found in the schema.' % feature_path)
      if f.type != schema_pb2.STRUCT:
        raise ValueError(
            'Step %s in feature %s does not refer to a valid STRUCT feature' %
            (step, feature_path))
      feature_container = f.struct_domain.feature

  feature = look_up_feature(feature_path.steps()[-1], feature_container)
  if feature is None:
    raise ValueError('Feature %s not found in the schema.' % feature_path)
  return feature


In [28]:
import argparse

import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

def get_feature(schema,feature_path):
    if not isinstance(schema, schema_pb2.Schema):
        raise TypeError('schema is of type %s, should be a Schema proto.' %
                    type(schema).__name__)

    if not isinstance(feature_path, types.FeaturePath):
        feature_path = types.FeaturePath([feature_path])
    
    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          
    
    with tf.io.gfile.GFile(feature_path, "r") as f:                                                                                                                                                                                                                     
        proto_str = f.read()
    print(schema)
    print(proto_str)

In [29]:
get_feature(schema_pb2.Schema,'configs/tf2/centernet_hourglass104_1024x1024_coco17_tpu-32.config')

TypeError: schema is of type GeneratedProtocolMessageType, should be a Schema proto.